# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments  \
0  Sep  9 2022  4:22PM  246095        12            HH19   
1  Sep  9 2022  4:19PM  246094        19  ACG-2102491198   
2  Sep  9 2022  4:05PM  246093        15      VT80177667   
3  Sep  9 2022  4:05PM  246093        15      VT80177722   
4  Sep  9 2022  4:05PM  246093        15      VT80177723   
5  Sep  9 2022  4:05PM  246093        15      VT80177724   
6  Sep  9 2022  4:05PM  246093        15      VT80177725   
7  Sep  9 2022  4:05PM  246093        15      VT80177727   
8  Sep  9 2022  3:59PM  246033        15      PNC387645A   
9  Sep  9 2022  3:59PM  246033        15      PNC387646A   

                      Customer ShipmentStatus  
0                    Hush Hush       Released  
1        ACG North America LLC       Released  
2  Virtus Pharmaceuticals, LLC       Released  
3  Virtus Pharmaceuticals, LLC      Executing  
4  Virtus Pharmaceuticals, LLC      Executing  
5  Virtus Pharmaceuticals, LLC       Released  
6  Virtus Pharmaceuticals, LLC      Executing  
7  Virtus Pharmaceuticals, LLC       Released  
8         Person & Covey, Inc.       Released  
9         Person & Covey, Inc.      Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
24  246092       Released          1
25  246093      Executing          3
26  246093       Released          3
27  246094       Released          1
28  246095       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
246088                1.0        4.0       3.0
246092                NaN        NaN       1.0
246093                NaN        3.0       3.0
246094                NaN        NaN       1.0
246095                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245733                0.0        1.0       0.0
245771                0.0        1.0       0.0
246008                5.0        0.0       5.0
246023               12.0        6.0      10.0
246031                0.0        0.0      16.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245733                  0          1         0
245771                  0          1         0
246008                  5          0         5
246023                 12          6        10
246031                  0          0        16

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               245733          0          1         0
1               245771          0          1         0
2               246008          5          0         5
3               246023         12          6        10
4               246031          0          0        16

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               245733                   1         
1               245771                   1         
2               246008         5                  5
3               246023        12         6       10
4               246031                           16

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Sep  9 2022  4:22PM  246095        12                       Hush Hush
1   Sep  9 2022  4:19PM  246094        19           ACG North America LLC
2   Sep  9 2022  4:05PM  246093        15     Virtus Pharmaceuticals, LLC
8   Sep  9 2022  3:59PM  246033        15            Person & Covey, Inc.
33  Sep  9 2022  3:52PM  246092        16           ACG North America LLC
34  Sep  9 2022  3:26PM  246088        10               ISDIN Corporation
42  Sep  9 2022  3:00PM  246083        19         GUSA Granules USA, Inc.
44  Sep  9 2022  2:54PM  246082        21               NBTY Global, Inc.
45  Sep  9 2022  2:37PM  246080        19         GUSA Granules USA, Inc.
46  Sep  9 2022  1:06PM  246076        16  Sartorius Bioprocess Solutions

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Sep  9 2022  4:22PM  246095        12                       Hush Hush   
1  Sep  9 2022  4:19PM  246094        19           ACG North America LLC   
2  Sep  9 2022  4:05PM  246093        15     Virtus Pharmaceuticals, LLC   
3  Sep  9 2022  3:59PM  246033        15            Person & Covey, Inc.   
4  Sep  9 2022  3:52PM  246092        16           ACG North America LLC   
5  Sep  9 2022  3:26PM  246088        10               ISDIN Corporation   
6  Sep  9 2022  3:00PM  246083        19         GUSA Granules USA, Inc.   
7  Sep  9 2022  2:54PM  246082        21               NBTY Global, Inc.   
8  Sep  9 2022  2:37PM  246080        19         GUSA Granules USA, Inc.   
9  Sep  9 2022  1:06PM  246076        16  Sartorius Bioprocess Solutions   

  Completed Executing Released  
0                            1  
1                            1  
2                   3        3  
3                  17        8  
4                            1  
5         1         4        3  
6                            2  
7                   1           
8                            1  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Sep  9 2022  4:22PM  246095        12                       Hush Hush   
1  Sep  9 2022  4:19PM  246094        19           ACG North America LLC   
2  Sep  9 2022  4:05PM  246093        15     Virtus Pharmaceuticals, LLC   
3  Sep  9 2022  3:59PM  246033        15            Person & Covey, Inc.   
4  Sep  9 2022  3:52PM  246092        16           ACG North America LLC   
5  Sep  9 2022  3:26PM  246088        10               ISDIN Corporation   
6  Sep  9 2022  3:00PM  246083        19         GUSA Granules USA, Inc.   
7  Sep  9 2022  2:54PM  246082        21               NBTY Global, Inc.   
8  Sep  9 2022  2:37PM  246080        19         GUSA Granules USA, Inc.   
9  Sep  9 2022  1:06PM  246076        16  Sartorius Bioprocess Solutions   

  Released Executing Completed  
0        1                      
1        1                      
2        3         3            
3        8        17            
4        1                      
5        3         4         1  
6        2                      
7                  1            
8        1                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Sep  9 2022  4:22PM  246095        12                    Hush Hush   
1  Sep  9 2022  4:19PM  246094        19        ACG North America LLC   
2  Sep  9 2022  4:05PM  246093        15  Virtus Pharmaceuticals, LLC   
3  Sep  9 2022  3:59PM  246033        15         Person & Covey, Inc.   
4  Sep  9 2022  3:52PM  246092        16        ACG North America LLC   

  Released Executing Completed  
0        1                      
1        1                      
2        3         3            
3        8        17            
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Sep  9 2022  4:22PM  246095        12                    Hush Hush   
1  Sep  9 2022  4:19PM  246094        19        ACG North America LLC   
2  Sep  9 2022  4:05PM  246093        15  Virtus Pharmaceuticals, LLC   
3  Sep  9 2022  3:59PM  246033        15         Person & Covey, Inc.   
4  Sep  9 2022  3:52PM  246092        16        ACG North America LLC   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       1.0        NaN        NaN  
2       3.0        3.0        NaN  
3       8.0       17.0        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Sep  9 2022  4:22PM  246095        12                    Hush Hush   
1  Sep  9 2022  4:19PM  246094        19        ACG North America LLC   
2  Sep  9 2022  4:05PM  246093        15  Virtus Pharmaceuticals, LLC   
3  Sep  9 2022  3:59PM  246033        15         Person & Covey, Inc.   
4  Sep  9 2022  3:52PM  246092        16        ACG North America LLC   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       1.0        0.0        0.0  
2       3.0        3.0        0.0  
3       8.0       17.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10          492135      17.0        4.0        1.0
12          246095       1.0        0.0        0.0
15          984165      32.0       20.0        5.0
16         1722500      11.0        1.0        0.0
19          738257       4.0        0.0        0.0
20          492076      11.0        6.0       12.0
21          737586       0.0        3.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10   492135      17.0        4.0        1.0
1        12   246095       1.0        0.0        0.0
2        15   984165      32.0       20.0        5.0
3        16  1722500      11.0        1.0        0.0
4        19   738257       4.0        0.0        0.0
5        20   492076      11.0        6.0       12.0
6        21   737586       0.0        3.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      17.0        4.0        1.0
1        12       1.0        0.0        0.0
2        15      32.0       20.0        5.0
3        16      11.0        1.0        0.0
4        19       4.0        0.0        0.0
5        20      11.0        6.0       12.0
6        21       0.0        3.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   17.0
1        12  Released    1.0
2        15  Released   32.0
3        16  Released   11.0
4        19  Released    4.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15    16   19    20   21
Status                                          
Completed   1.0  0.0   5.0   0.0  0.0  12.0  0.0
Executing   4.0  0.0  20.0   1.0  0.0   6.0  3.0
Released   17.0  1.0  32.0  11.0  4.0  11.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15    16   19    20   21
0          Completed   1.0  0.0   5.0   0.0  0.0  12.0  0.0
1          Executing   4.0  0.0  20.0   1.0  0.0   6.0  3.0
2           Released  17.0  1.0  32.0  11.0  4.0  11.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15    16   19    20   21
0  Completed   1.0  0.0   5.0   0.0  0.0  12.0  0.0
1  Executing   4.0  0.0  20.0   1.0  0.0   6.0  3.0
2   Released  17.0  1.0  32.0  11.0  4.0  11.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()